# Data Processing Pipeline
This notebook is an interactive version of the data processing files, and also allows to explore the dataset interactively.

## Create Dataset
This script assume the dataset has already been downloaded

Since the sentences are placed in different files in the source, we haev to match each row from different language files using ids files.

In [10]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from tqdm import tqdm

def load_text_file(file_path):
    """Load a text file and return a list of lines."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def load_ids_file(file_path):
    """
    Load an .ids file and return a list of mappings.
    
    Each line in the file is expected to have the format:
    source_file\ttarget_file\tsource_positions\ttarget_positions
    
    Returns:
        List of dictionaries with parsed mapping information
    """
    mappings = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            parts = line.strip().split('\t')
            if len(parts) == 4:
                mapping = {
                    'source_file': parts[0],
                    'target_file': parts[1],
                    'source_positions': parts[2].split(),
                    'target_positions': parts[3].split()
                }
                mappings.append(mapping)
            else:
                print(f"Warning: Skipping malformed line in {file_path}: {line.strip()}")
    return mappings

def create_north_levantine_dataset(data_dir):
    """
    Create a Hugging Face dataset from the North Levantine parallel corpus.
    
    Args:
        data_dir: Directory containing the dataset files
    
    Returns:
        A DatasetDict containing the aligned parallel corpus
    """
    # ISO codes for all languages in the dataset
    languages = ['eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa']
    
    # Step 1: Discover and load all language files
    text_data = {}
    
    # Find all language files in the directory
    language_files = {}
    for file in tqdm(os.listdir(data_dir), desc="Finding languages"):
        if file.endswith(tuple(languages)) and not file.endswith(".ids"):
            lang_code = file.split(".")[-1]
            language_files[lang_code] = os.path.join(data_dir, file)
    
    # Load discovered language files
    for lang in tqdm(languages, desc="Loading language files"):
        if lang in language_files:
            text_data[lang] = load_text_file(language_files[lang])
            print(f"Loaded {lang} file with {len(text_data[lang])} lines")
        else:
            print(f"Warning: File for language '{lang}' not found in {data_dir}")
    
    # Step 2: Create individual datasets with index column
    language_datasets = {}
    for lang, lines in tqdm(text_data.items(),desc="Creating individual datasets"):
        df = pd.DataFrame({
            'text': lines,
            'index': list(range(len(lines)))
        })
        language_datasets[lang] = Dataset.from_pandas(df)
    
    # Step 3: Create a merged dataset
    # Start with English dataset as base
    if 'eng' not in language_datasets:
        raise ValueError("English dataset is required as the base for alignment")
    
    # Get the number of examples in the English dataset
    num_examples = len(language_datasets['eng'])
    
    # Verify all languages have the same number of examples
    for lang, dataset in language_datasets.items():
        if len(dataset) != num_examples:
            print(f"Warning: {lang} dataset has {len(dataset)} examples, but English has {num_examples}")
    
    # Create a dictionary for the merged dataset
    merged_data = {
        'line_idx': list(range(num_examples)),
        'eng': language_datasets['eng']['text']
    }
    
    # Add other languages
    for lang in languages:
        if lang != 'eng' and lang in language_datasets:
            if len(language_datasets[lang]) == num_examples:
                merged_data[lang] = language_datasets[lang]['text']
            else:
                # Handle mismatched sizes by padding with empty strings
                padded_texts = language_datasets[lang]['text'] + [''] * (num_examples - len(language_datasets[lang]))
                merged_data[lang] = padded_texts[:num_examples]
    
    # Create the merged dataset
    merged_dataset = Dataset.from_dict(merged_data)
    
    # Create a dataset dictionary with train split
    dataset_dict = DatasetDict({
        'train': merged_dataset
    })
    
    return dataset_dict

def get_parallel_sentences(dataset, index, languages=None):
    """
    Get parallel sentences for a specific example across all or specified languages.
    
    Args:
        dataset: The dataset containing parallel sentences
        index: The index of the example to retrieve
        languages: Optional list of language codes to include (default: all available)
    
    Returns:
        Dictionary mapping language codes to sentences
    """
    if index >= len(dataset) or index < 0:
        raise ValueError(f"Index {index} out of range (0-{len(dataset)-1})")
    
    example = dataset[index]
    
    if languages is None:
        # Get all available languages excluding non-text columns
        languages = [col for col in example.keys() if col not in ['line_idx', 'index']]
    
    return {lang: example[lang] for lang in languages if lang in example}


In [11]:

data_dir = ".\\..\\..\\data\\dataset\\UFAL Parallel Corpus of North Levantine 1.0"

# Create the dataset
dataset = create_north_levantine_dataset(data_dir)


Loading language files:  43%|████▎     | 3/7 [00:00<00:00, 19.30it/s]

Loaded eng file with 120600 lines
Loaded arb file with 120600 lines
Loaded apc file with 120600 lines
Loaded deu file with 120600 lines
Loaded ell file with 120600 lines


Loading language files: 100%|██████████| 7/7 [00:00<00:00, 20.39it/s]


Loaded fra file with 120600 lines
Loaded spa file with 120600 lines


Creating individual datasets: 100%|██████████| 7/7 [00:00<00:00,  9.99it/s]


## View Dataset

In [12]:
dataset["train"][0]

{'line_idx': 0,
 'eng': "Let's talk about the time Moldova made Romania a birthday cake and Romania said it tasted good even though it didn't.",
 'arb': 'دعونا نتحدّث عن الوقت الذي (قدّمت فيه (مالدوفا) لـ(رومانيا ،كعكة عيد ميلاد و (رومانيا) قالت أنّ طعمها جيّد حتّى و إن لم تكن كذلك',
 'apc': 'خلونا نحكي عن الوقت اللي قدمت فيه مالدوفا لرومانيا، كعكة عيد ميلاد ورومانيا قالت إنو طيبة حتى لو ما كانت طيبة',
 'deu': 'Reden wir darüber, als Moldavien Rumänien einen Geburtstagskuchen machte und Rumänien meinte, er wäre lecker, obwohl er das überhaupt nicht war.',
 'ell': 'Ας μιλήσουμε για το όταν η Μολδαβία έκανε στη Ρουμανία μια τούρτα και η Ρουμανία είπε ότι είναι νόστιμη, αν και δεν ήταν.',
 'fra': "Parlons de l'époque Moldova a fait la Roumanie un gâteau d'anniversaire et de la Roumanie a déclaré qu'il avait bon goût même si elle n'a pas fait.",
 'spa': 'Hablemos de la vez que Moldavia le hizo a Rumanía un pastel de cumpleaños y Rumanía dijo que sabía bien aunque no era verdad.'}

In [14]:
# Print some information about the dataset
print("\nDataset Information:")
print(f"Number of examples: {len(dataset['train'])}")
print(f"Features: {dataset['train'].features}")

# Show some examples
print(f"\nShowing {5} random examples:")
import random
sample_indices = random.sample(range(len(dataset['train'])), min(5, len(dataset['train'])))

for i, idx in enumerate(sample_indices):
    parallel_sentences = get_parallel_sentences(dataset['train'], idx,["eng","arb","apc"])
    print(f"\nExample {i+1} (Index {idx}):")
    for lang, sentence in parallel_sentences.items():
        print(f"  {lang}: {sentence}")




Dataset Information:
Number of examples: 120600
Features: {'line_idx': Value(dtype='int64', id=None), 'eng': Value(dtype='string', id=None), 'arb': Value(dtype='string', id=None), 'apc': Value(dtype='string', id=None), 'deu': Value(dtype='string', id=None), 'ell': Value(dtype='string', id=None), 'fra': Value(dtype='string', id=None), 'spa': Value(dtype='string', id=None)}

Showing 5 random examples:

Example 1 (Index 62789):
  eng: Well, even using the ring will take its toll.
  arb: حتى استخدام الخاتم لن يفيدك
  apc: حتى استخدام الخاتم مارح يفيدك.

Example 2 (Index 38197):
  eng: It's good to see that not everybody's put off their dinner by the weather forecast.
  arb: إنه من الجيّد رؤية ليس هناك أحد يترك عشائه جراء الأرصاد الجوية.
  apc: منيح شوفة ما في حدا بيترك العشا تبعو بسبب الأرصاد الجوية.

Example 3 (Index 79060):
  eng: Someone on our team took a laptop from Mark's safe house.
  arb: شخصٌ ما في فريقنا أخذ حاسوب محمول من منزل (مارك) الأمن
  apc: شخص بفريقنا أخد حاسوب من بيت ما

## Save to disk

In [69]:
dataset = dataset["train"].train_test_split(test_size=0.15, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['line_idx', 'eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa'],
        num_rows: 102510
    })
    test: Dataset({
        features: ['line_idx', 'eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa'],
        num_rows: 18090
    })
})

In [68]:
# Save the dataset locally
dataset.save_to_disk("../../Data/UFAL Parallel Corpus of North Levantine 1.0/Processed")


Saving the dataset (1/1 shards): 100%|██████████| 18090/18090 [00:00<00:00, 71924.37 examples/s]


## Transliterate dataset

In [61]:
from camel_tools.utils.charmap import CharMapper

sentence = "ذهبت إلى المكتبة."
print(sentence)

ar2bw = CharMapper.builtin_mapper('ar2hsb')
bw2ar = CharMapper.builtin_mapper('hsb2ar')

sent_bw = ar2bw(sentence)
sent_ar = bw2ar(sent_bw)
print(sent_bw)
print(sent_ar)


ذهبت إلى المكتبة.
ðhbt Ălý Almktbħ.
ذهبت إلى المكتبةْ


In [52]:
def process_row(row):
    return {"eng":row["eng"], "arb": [ar2bw(ar) for ar in row["arb"]] if isinstance(row["arb"], list) else ar2bw(row["arb"])}
    

In [56]:
dataset_test = dataset.map(process_row,remove_columns=dataset["train"].column_names,batched=True,batch_size=128)

Map: 100%|██████████| 120600/120600 [00:01<00:00, 92329.21 examples/s]


In [57]:
dataset_test["train"][0]

{'eng': "Let's talk about the time Moldova made Romania a birthday cake and Romania said it tasted good even though it didn't.",
 'arb': 'dEwnA ntHd~v En Alwqt Al*y (qd~mt fyh (mAldwfA) l_(rwmAnyA ،kEkp Eyd mylAd w (rwmAnyA) qAlt >n~ TEmhA jy~d Ht~Y w <n lm tkn k*lk'}

## Something else

In [88]:
def format_fn(row):
    comp = [row["eng"],ar2bw(row["arb"],ar2bw(row["apc"]))] if not isinstance(row["arb"], list) else row["eng"]+[ar2bw(ar) for ar in row["arb"]]+[ar2bw(ar) for ar in row["apc"]]
    promp = [""]*len(comp)
    return {"prompt":promp,"completion":comp }

In [52]:
def format_fn(row):
    if not isinstance(row["arb"], list):
        eng_prompt = [f"english:{row["eng"]}\n"]
        arb_prompt = [f"classical arabic:{ar2bw(row["arb"])}\n"]
        apc_prompt = [f"levantine arabic:{ar2bw(row["apc"])}\n"]
    else:
        eng_prompt = [f"english:{x}\n" for x in row["eng"]]
        arb_prompt = [f"classical arabic:{ar2bw(x)}\n" for x in row["arb"]]
        apc_prompt = [f"levantine arabic:{ar2bw(x)}\n" for x in row["apc"]]
    eng_comp = ["classical arabic:","levantine arabic:"]  * len(eng_prompt)
    arb_comp =  ["english:","levantine arabic:"] * len(eng_prompt) 
    apc_comp = ["english:","classical arabic:"] * len(eng_prompt)

    prompts = eng_prompt*2+arb_prompt*2+apc_prompt*2
    completions = eng_comp+arb_comp+apc_comp
    return {"prompt":prompts,"completion":completions }

In [53]:
dataset_test_2 = dataset.map(format_fn,remove_columns=dataset["train"].column_names,batched=True,batch_size=128)

Map: 100%|██████████| 18090/18090 [00:00<00:00, 35446.76 examples/s]


In [58]:
dataset_test_2["train"][3]

{'prompt': 'english:Massive tits, never speaks.\n',
 'completion': 'levantine arabic:'}

In [92]:
dataset_test_2["train"]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 307530
})

In [59]:

dataset_test_2.push_to_hub("KHuss/UFAL_levantine_buckwater_seq2seq",private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/KHuss/UFAL_levantine_buckwater_seq2seq/commit/a4a4629be9d3cdcc8ff315c33ef3a2e9a5933079', commit_message='Upload dataset', commit_description='', oid='a4a4629be9d3cdcc8ff315c33ef3a2e9a5933079', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KHuss/UFAL_levantine_buckwater_seq2seq', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KHuss/UFAL_levantine_buckwater_seq2seq'), pr_revision=None, pr_num=None)